# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
import getpass
myname = getpass.getuser().split('-')[1]
myname
#MYTABLE=myname + '_data'

'ai2z'

In [2]:
#alternate name info
myname2 = 'pbstudent'
MYTABLE= myname2 + '_data'
MYTABLE

'pbstudent_data'

In [3]:
#Connection
import os
import pymysql #for accessing the mySQL database
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

#note to self: no need to change the user info above

In [4]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [5]:
#read in data (csv format)
esri_url = 'https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326'
df = pd.read_csv(esri_url)
df.head()

,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,Business_ZIP_Code,...,Payment_Year_Number,Program_Type,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year
0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,95501,...,4,Medicare/Medicaid,Acute Care Hospitals,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016
1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,94545,...,4,Medicare/Medicaid,Acute Care Hospitals,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016
2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,94558,...,4,Medicare/Medicaid,Acute Care Hospitals,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016
3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,95642,...,4,Medicare/Medicaid,Acute Care Hospitals,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019
4,-122.435820,37.769049,5,Sutter Bay Hospitals,1659439834,50008,CASTRO AND DUBOCE,San Francisco,San Francisco,94114,...,4,Medicare/Medicaid,Acute Care Hospitals,984465.12,MU,98446.51,37.769049,-122.435820,2016,2019


In [6]:
#write to database table using sql
df.to_sql(name=MYTABLE, con=conn, index=True, if_exists='replace')

#Parameter Descriptions
#name => Name of SQL table
#con => database connection (e.g. sqlalchemy.engine)
#if_exists => what to do if the table already exists (options include:{‘fail’, ‘replace’, ‘append’}, default ‘fail’)
#---> fail: Raise a ValueError.; replace: Drop the table before inserting new values.; append: Insert new values to the existing table.
#indexbool => Write DataFrame index as a column. Uses index_label as the column name in the table (default is True)

#check table names
#pd.read_sql_query("SHOW TABLES", conn)
df_dbshow = pd.read_sql_query("SHOW TABLES", conn)
# df_dbshow

#yes! the data exists (11 	pbstudent_data)

In [7]:
#try to read back the data
df_MYTABLE = pd.read_sql_query("SELECT * FROM " + MYTABLE, conn)
#df_MYTABLE.head()
#it works!!! Super!

In [8]:
#some quick checks...
df_esri = pd.read_sql_query("SELECT * FROM " + MYTABLE, conn)
print('The Shape of the dowloaded ESRI file is:', df_esri.shape)
print('columns =', df_esri.columns)
list(df_esri.columns) == ['X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year']

#got it now!, I used index=False in the write.sql syntax above... need to fix this to the default=True for the assertion below to work

The Shape of the dowloaded ESRI file is: (323, 22)
columns = Index(['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'],
      dtype='object')


False

In [9]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
# assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [10]:
#quick checks
print('The Shape of the dowloaded ESRI file is:', df_esri.shape)
print('columns =', df_esri.columns)

The Shape of the dowloaded ESRI file is: (323, 22)
columns = Index(['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'],
      dtype='object')


In [11]:
#QUESTION: Which providers are located in the zipcode with the largest population?

#first, let's take a peek at the population table
population = pd.read_sql_query("SELECT * FROM population", conn)

print('The Shape of the population data file is:', population.shape)
print('columns =', population.columns)
population.head()

The Shape of the population data file is: (1650, 3)
columns = Index(['index', 'Zip', 'Population'], dtype='object')


,index,Zip,Population
0,0,90011,110750
1,1,90650,104524
2,2,91331,103683
3,3,90201,100512
4,4,90044,99980


In [12]:
#quick checks (check the shape of the merged table)
mytbl_popn = pd.read_sql_query("""
    SELECT distinct mytbl.*, pop.Zip, pop.Population
    FROM pbstudent_data  mytbl
      INNER JOIN population pop
        ON mytbl.Business_ZIP_Code = pop.Zip
""", con=conn)

print(mytbl_popn.shape)
mytbl_popn.head()

(317, 24)


,index,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,...,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year,Zip,Population
0,275,-118.099150,33.912613,276,"CPH HOSPITAL MANAGEMENT, LLC",1194016923,50771,13100 Studebaker Rd,Norwalk,Los Angeles,...,Acute Care Hospitals,2184880.10,MU,218488.01,33.912613,-118.099150,2018,2020,90650,104524
1,74,-117.432081,34.071840,75,KAISER FOUNDATION HOSPITALS,1356496772,50140,9961 Sierra Avenue,Fontana,San Bernardino,...,Acute Care Hospitals,1108375.17,MU,110837.52,34.071840,-117.432081,2018,2020,92335,99242
2,21,-118.445983,34.324488,22,COUNTY OF LOS ANGELES,1134143415,50040,14445 Olive View Drive,Sylmar,Los Angeles,...,Acute Care Hospitals,9246604.47,MU,924660.45,34.324488,-118.445983,2018,2019,91342,94432
3,48,-117.439101,33.926868,49,DOCTORS HOSPITAL OF RIVERSIDE LLC,1124596382,50102,3865 Jackson St,Riverside,Riverside,...,Acute Care Hospitals,4212811.78,MU,421281.18,33.926868,-117.439101,2018,2020,92503,93032
4,217,-117.685655,34.026272,218,"VERITAS HEALTH SERVICES, INC.",1962407460,50586,5451 Walnut Ave,Chino,San Bernardino,...,Acute Care Hospitals,1926048.03,MU,192604.80,34.026272,-117.685655,2014,2015,91710,90481


In [13]:
#Anwer the question using SQL query...
"""Answer the question: Which providers are located in the zipcode with the largest population?"""

"""Instructions: 
(1) In the database, there is an existing table called population. We want to merge the DHCS datafile 
loaded above with the population data available in this other database table. The tables can be merged 
on MYTABLE's Business_ZIP_Code field and population's Zip field.

(2) Note that not all Zip_Codes from your downloaded file have to be in the population table. 
If they aren't, then I want you to eliminate the non-matching records. 
That is, only keep the records that have a matching ZIP code in both sets of data.
"""

#It seems the second instruction is calling for an inner join...

#Quick checks
print(population.dtypes['Zip'])
print(population.columns)

int64
Index(['index', 'Zip', 'Population'], dtype='object')


In [14]:
#Option1: Do this in steps/stages
#1.1. join datasets & limit to max population
#Also use a "having" statement to limit to the zip code with the max population...
max_population = pd.read_sql_query("""
    SELECT distinct mytbl.*, pop.Zip, pop.Population
    FROM pbstudent_data  mytbl
      INNER JOIN population pop
        ON mytbl.Business_ZIP_Code = pop.Zip
    HAVING max(Population)
""", con=conn)
max_population

#1.2. get the zip for the high population
max_pop_zip = pd.read_sql_query("""
    SELECT distinct mytbl.Business_ZIP_Code, pop.Zip, pop.Population
    FROM pbstudent_data  mytbl
      INNER JOIN population pop
        ON mytbl.Business_ZIP_Code = pop.Zip
    HAVING max(Population)
""", con=conn)
max_pop_zip

#1.3. next, get the zipcode 
zipcode = max_population["Zip"][0]
zipcode

# Create the pandas DataFrame
# initialize list of zipcode(s) & create dataframe
data = [[zipcode]]
df_zip = pd.DataFrame(data, columns = ['ZipCode'])

#write the zip code info to the database
MY_ZipTABLE= myname2 + '_ZipCode'
df_zip.to_sql(name=MY_ZipTABLE, con=conn, index=True, if_exists='replace')

#try to read back the data (to check that it works...)
df_MY_ZipTABLE = pd.read_sql_query("SELECT * FROM " + MY_ZipTABLE, conn)

#1.4. next, get the list of providers 
#create the list of providers
providers = pd.read_sql_query("""
    SELECT distinct a.NPI
    FROM pbstudent_data  a
    LEFT join pbstudent_ZipCode b
        on a.Business_ZIP_Code =b.ZipCode
    where a.Business_ZIP_Code =b.ZipCode
""", con=conn)

providers

NPI_list=list(map(str, providers["NPI"]))
NPI_list


['1194016923']

In [15]:
#OR 
#Option2: Just select the providers in the zipcode with the max_population straigthaway using a subquery...see code chunk below
#Also use a "having" statement to limit to the zip code with the max population...

#next, get the zipcode & list of providers (used this one!!!)
#filter out the providers from this zip code using a subquery...
max_pop_providers = pd.read_sql_query("""
    SELECT distinct mytbl.NPI
    FROM pbstudent_data  mytbl
      INNER JOIN population pop
        ON mytbl.Business_ZIP_Code = pop.Zip
    WHERE Zip in 
        (SELECT distinct mytbl.Business_ZIP_Code
         FROM pbstudent_data  mytbl
            INNER JOIN population pop
            ON mytbl.Business_ZIP_Code = pop.Zip
         HAVING max(Population)
         )
""", con=conn)
max_pop_providers

#Final answer
NPI_list=list(map(str, max_pop_providers["NPI"]))
NPI_list

['1194016923']

In [16]:
#Final answer
answer=NPI_list
answer

['1194016923']

In [17]:
assert(type(answer) == list)
assert(answer == ['1194016923'])


## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [18]:
#first some quick checks...
#before the merge (i.e. df_MYTABLE)
df_esri['Medicaid_EP_Hospital_Type'].value_counts()

Acute Care Hospitals    312
Children's Hospitals     11
Name: Medicaid_EP_Hospital_Type, dtype: int64

In [19]:
#after the merge
mytbl_popn['Medicaid_EP_Hospital_Type'].value_counts()

Acute Care Hospitals    306
Children's Hospitals     11
Name: Medicaid_EP_Hospital_Type, dtype: int64

In [20]:
#Avg_Pay_per_Capita by Hospital Type
#compute the average Pay per_Capita (using sQL from start to finish)
"""1. From our downloaded data file, compute the Total payments per ZIP code and Medicaid EP Hospital Type."""

"""2. Then merge that with the population data to compute a Total payments per person."""

"""3. Then average that across all of the Medicaid EP Hospital Types to get 
an average per persona payment for these type of hospital.
Your answer should be in structure of a data frame with at least two columns:
    Medicaid_EP_Hospital_Type
    Avg_Pay_per_Capita
"""

Avg_Pay_per_Capita = pd.read_sql_query("""
    SELECT distinct df.Medicaid_EP_Hospital_Type
        ,avg(pp.tot_per_person_payments) as Avg_Pay_per_Capita
    FROM pbstudent_data df
    LEFT JOIN
        (
        SELECT distinct mytbl.Business_ZIP_Code, mytbl.Medicaid_EP_Hospital_Type
            ,a.totalPayments/b.Population as tot_per_person_payments, b.Population    
        FROM pbstudent_data  mytbl
        LEFT JOIN
            (SELECT distinct esri.Business_ZIP_Code, esri.Medicaid_EP_Hospital_Type
            ,sum(total_payments) as totalPayments
            FROM pbstudent_data esri
            GROUP BY esri.Business_ZIP_Code, esri.Medicaid_EP_Hospital_Type
            )a
            ON mytbl.Business_ZIP_Code = a.Business_ZIP_Code
            AND mytbl.Medicaid_EP_Hospital_Type = a.Medicaid_EP_Hospital_Type
        LEFT JOIN 
            (
            SELECT distinct pop.Zip, pop.Population
            FROM population pop
            ) b
           ON mytbl.Business_ZIP_Code = b.Zip       
        GROUP BY mytbl.Business_ZIP_Code
        )pp
    ON df.Medicaid_EP_Hospital_Type = pp.Medicaid_EP_Hospital_Type
    GROUP BY df.Medicaid_EP_Hospital_Type        
    """, con=conn)

#Note: the alias pp => per-person pay
Avg_Pay_per_Capita.head()

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,468.554016


In [21]:
#Not sure why the Childrens' hospital value is failing the assertion (468.554 vs. 329.551)? Ok - let's do this piece-meal...

In [22]:
#Total by Hospital Type

#1.Aggregate total payments by zip...
"""1. From our downloaded data file, compute the Total payments per ZIP code and Medicaid EP Hospital Type."""

hosp_pay_zip = pd.read_sql_query("""
    SELECT distinct mytbl.Business_ZIP_Code, mytbl.Medicaid_EP_Hospital_Type
    ,sum(total_payments) as sum_total_payments
    FROM pbstudent_data  mytbl
    GROUP BY mytbl.Business_ZIP_Code, mytbl.Medicaid_EP_Hospital_Type
    """, con=conn)
print(hosp_pay_zip.shape)
#hosp_pay_zip.head()

(267, 3)


In [23]:
#2. Merge total payments by zip to pupulation data & compute Total payments per person

"""2. Then merge that with the population data to compute a Total payments per person."""
#first, write the hosp_pay_zip data to the database
df_HospPay= myname2 + '_HospPay'
hosp_pay_zip.to_sql(name=df_HospPay, con=conn, index=True, if_exists='replace')

pd.read_sql_query('SHOW TABLES', conn)
# #try to read back the data (to check that it works...)
df_HospPay_zip = pd.read_sql_query("SELECT * FROM " + df_HospPay, conn)
df_HospPay_zip.head()

#next, merge hosp_pay with population data
person_pay_zip = pd.read_sql_query("""
    SELECT distinct a.*, b.*
        ,a.sum_total_payments/b.Population as tot_per_person_payments  
    FROM pbstudent_HospPay a
    LEFT JOIN population b
        ON a.Business_ZIP_Code = b.Zip       
    GROUP BY a.Business_ZIP_Code
    
   """, con=conn)
print('The Shape of the Person_Pay_by_Zip Summary file is:', person_pay_zip.shape)
#person_pay_zip.head()

The Shape of the Person_Pay_by_Zip Summary file is: (262, 8)


In [24]:
#Comments: It seems like the problem occurs at step 2... the dataframe shape goes from 267 rows to 262 rows (losing 5 observations)

In [25]:
#3. Average Total payments per person
"""3. Then average that across all of the Medicaid EP Hospital Types to get 
an average per persona payment for these type of hospital.
Your answer should be in structure of a data frame with at least two columns:
    Medicaid_EP_Hospital_Type
    Avg_Pay_per_Capita
"""
#As before, first write the person_pay_zip data to the database
df_PersonPay= myname2 + '_PersonPay'
person_pay_zip.to_sql(name=df_PersonPay, con=conn, index=True, if_exists='replace')

pd.read_sql_query('SHOW TABLES', conn)
# #try to read back the data (to check that it works...)
df_PersonPay_zip = pd.read_sql_query("SELECT * FROM " + df_PersonPay, conn)
df_PersonPay_zip.head()

#next, do the average...
Avg_Pay_per_Capita = pd.read_sql_query("""
    SELECT distinct Medicaid_EP_Hospital_Type
        ,avg(tot_per_person_payments) as Avg_Pay_per_Capita
    FROM pbstudent_PersonPay 
    GROUP BY Medicaid_EP_Hospital_Type        
    """, con=conn)

print(Avg_Pay_per_Capita.shape)
Avg_Pay_per_Capita.head()

(2, 2)


,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,468.554016


In [26]:
answer=Avg_Pay_per_Capita
answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,468.554016


In [27]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
# assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)

In [28]:
#The last assertion test/statemnet above is failing (468.554 vs. 329.551), as a result of the dropped zipcode cases i.e.
#where some of the zipcodes in the Hospital Pay Summary data from the downloaded ESRI file are not in the population data...
#(see addendum below)...

## Now using Pandas instead of SQL

In [29]:
df_MYTABLE.columns

Index(['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'],
      dtype='object')

In [30]:
df_MYTABLE2 = df_MYTABLE[["Business_ZIP_Code", "Medicaid_EP_Hospital_Type", "total_payments"]]
print(df_MYTABLE2.shape)
df_MYTABLE2.head()

(323, 3)


,Business_ZIP_Code,Medicaid_EP_Hospital_Type,total_payments
0,95501,Acute Care Hospitals,1530950.70
1,94545,Acute Care Hospitals,3245920.28
2,94558,Acute Care Hospitals,1262015.89
3,95642,Acute Care Hospitals,865130.13
4,94114,Acute Care Hospitals,984465.12


In [31]:
#Avg_Pay_per_Capita by Hospital Type
#compute the average Pay per_Capita (using pandas from start to finish)
"""1. From our downloaded data file, compute the Total payments per ZIP code and Medicaid EP Hospital Type."""
#first reduce the dimension to see only the relevant columns 
df_MYTABLE2 = df_MYTABLE[["Business_ZIP_Code", "Medicaid_EP_Hospital_Type", "total_payments"]]
df_MYTABLE2

#totalpay_zips = df_MYTABLE.groupby(['Business_ZIP_Code','Medicaid_EP_Hospital_Type']).agg({'total_payments': ['sum']})
totalpay_zips = df_MYTABLE2.groupby(["Business_ZIP_Code","Medicaid_EP_Hospital_Type"]).sum("total_payments").reset_index()
#totalpay_zips = df_MYTABLE2.groupby(by=['Business_ZIP_Code','Medicaid_EP_Hospital_Type']).sum().reset_index()

print('The Shape of Total_pay_byZip is:', totalpay_zips.shape)
totalpay_zips.head()

"""2. Then merge that with the population data to compute a Total payments per person."""
df_merge = totalpay_zips.merge(population, left_on="Business_ZIP_Code", right_on="Zip",  how='left')

#compute a Total payments per person
df_merge["totalpay_per_person"] = df_merge["total_payments"]/df_merge["Population"] #totalpay_per_person

print('The Shape of Total_pay_byZip Merged with Population data is:', df_merge.shape)
df_merge.head()

"""3. Then average that across all of the Medicaid EP Hospital Types to get 
an average per person payment for these type of hospital.
Your answer should be in structure of a data frame with at least two columns:
    Medicaid_EP_Hospital_Type
    Avg_Pay_per_Capita
"""
#get the average by hospital type
# avg_person_pay_byzip = df_merge.groupby(['Medicaid_EP_Hospital_Type']).agg({'totalpay_per_person': ['mean']})
avg_person_pay_byzip = df_merge.groupby(by=['Medicaid_EP_Hospital_Type'])['totalpay_per_person'].mean()
avg_person_pay_byzip

avg_df = avg_person_pay_byzip.reset_index()
avg_df.columns = ['Medicaid_EP_Hospital_Type', 'Avg_Pay_per_Capita']

The Shape of Total_pay_byZip is: (267, 3)
The Shape of Total_pay_byZip Merged with Population data is: (267, 7)


In [32]:
#final answer
answer=avg_df
answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,329.550946


In [33]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)

#### ADDENDUM (Deep dive into the dropped  Zipcode cases...)

In [34]:
#Addendum...
#Deep dive into the dropped cases...

In [35]:
#quick checks...
print(hosp_pay_zip.shape)
hosp_pay_zip.head()

(267, 3)


,Business_ZIP_Code,Medicaid_EP_Hospital_Type,sum_total_payments
0,90004,Acute Care Hospitals,1093021.73
1,90012,Acute Care Hospitals,3614739.93
2,90015,Acute Care Hospitals,10045484.04
3,90017,Acute Care Hospitals,3899845.21
4,90020,Children's Hospitals,427640.22


In [36]:
#quick checks...
print(person_pay_zip.shape)
person_pay_zip.head()

(262, 8)


,index,Business_ZIP_Code,Medicaid_EP_Hospital_Type,sum_total_payments,index,Zip,Population,tot_per_person_payments
0,0,90004,Acute Care Hospitals,1093021.73,115.0,90004.0,61105.0,17.887599
1,1,90012,Acute Care Hospitals,3614739.93,433.0,90012.0,37268.0,96.993129
2,2,90015,Acute Care Hospitals,10045484.04,738.0,90015.0,23900.0,420.313140
3,3,90017,Acute Care Hospitals,3899845.21,648.0,90017.0,27832.0,140.120912
4,4,90020,Children's Hospitals,427640.22,390.0,90020.0,39366.0,10.863187


In [37]:
#check the dropped records...
dropped_cases = pd.read_sql_query("""
    SELECT a.*
    FROM pbstudent_HospPay a
    WHERE a.Business_ZIP_Code not in
        (SELECT Zip
        FROM population)    
   """, con=conn)
print(dropped_cases.shape)
dropped_cases

(6, 4)


,index,Business_ZIP_Code,Medicaid_EP_Hospital_Type,sum_total_payments
0,17,90095,Acute Care Hospitals,4069740.52
1,28,90505,Acute Care Hospitals,1626121.80
2,36,90720,Acute Care Hospitals,929043.20
3,104,92415,Acute Care Hospitals,10325565.88
4,177,94143,Acute Care Hospitals,4440113.87
5,246,95825,Acute Care Hospitals,350055.79


In [38]:
#Aha! It seems the one workaround on this issue of dropped records may be to use a full outer join in the SQL scenario (that's for another time...)!!!
#Buy I still don't get how the pandas (non-sql approach) worked, even though I used the same left join in that scenario!!! 
# A Mystery, I guess...

In [39]:
#Clean Up Time!!!

In [40]:
with conn.connect() as c:
    c.execute('DROP TABLE IF EXISTS  pbstudent_HospPay, pbstudent_PersonPay, pbstudent_ZipCode, pbstudent_data')

In [41]:
#pd.read_sql_query('SHOW TABLES', conn)



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')